In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import os
import pickle

In [2]:
nfl_df = pd.read_csv('Data/NFL_Predictions_df.csv')
print(nfl_df.shape)
nfl_df.head(5)


(544, 23)


,Team,Week,Result,Record,Home,Opponent,Points Scored,Points Allowed,1st Downs,Total Yards Gained,...,1st Downs Allowed,Total Yards Allowed,Passing Yards Allowed,Rushing Yards Allowed,Turnovers Gained,Offense Expected Points,Defense Expected Points,Special Teams Expected Points,Prediction_LR,Prediction_ADA
0,San Francisco 49ers,1,1,1-0,0,Pittsburgh Steelers,30,7,22,391,...,15,239,198,41,2.0,9.73,13.55,-0.78,0.407722,0.033581
1,San Francisco 49ers,2,1,2-0,0,Los Angeles Rams,30,23,21,365,...,28,386,297,89,2.0,12.36,-3.31,-2.05,0.407722,0.033581
2,San Francisco 49ers,3,1,3-0,1,New York Giants,30,12,26,441,...,10,150,121,29,1.0,19.00,11.82,-9.72,0.407722,0.033581
3,San Francisco 49ers,4,1,4-0,1,Arizona Cardinals,35,16,30,395,...,23,362,257,105,1.0,30.46,-9.72,-1.63,0.407722,0.033581
4,San Francisco 49ers,5,1,5-0,1,Dallas Cowboys,42,10,25,421,...,8,197,140,57,4.0,12.84,23.12,-5.21,0.407722,0.033581


In [3]:
print(nfl_df.columns)

Index(['Team', 'Week', 'Result', 'Record', 'Home', 'Opponent', 'Points Scored',
       'Points Allowed', '1st Downs', 'Total Yards Gained', ' Passing Yards',
       ' Rushing Yards', 'Turnovers Lost', '1st Downs Allowed',
       'Total Yards Allowed', 'Passing Yards Allowed', 'Rushing Yards Allowed',
       'Turnovers Gained', 'Offense Expected Points',
       'Defense  Expected Points', 'Special Teams  Expected Points',
       'Prediction_LR', 'Prediction_ADA'],
      dtype='object')


In [4]:
nfl_df.rename(columns={
    'Passing Yards': 'O_pass_yards',
    'Rushing Yards': 'O_rush_yards'
}, inplace=True)

In [5]:
print(nfl_df.columns)

Index(['Team', 'Week', 'Result', 'Record', 'Home', 'Opponent', 'Points Scored',
       'Points Allowed', '1st Downs', 'Total Yards Gained', ' Passing Yards',
       ' Rushing Yards', 'Turnovers Lost', '1st Downs Allowed',
       'Total Yards Allowed', 'Passing Yards Allowed', 'Rushing Yards Allowed',
       'Turnovers Gained', 'Offense Expected Points',
       'Defense  Expected Points', 'Special Teams  Expected Points',
       'Prediction_LR', 'Prediction_ADA'],
      dtype='object')


In [6]:
# get index of team data
team_index = nfl_df['Team']

# remove opponent data for mlp model prediction results
mlp_model_df = nfl_df.drop(columns=['Week', 'Result', 'Record', 'Opponent', 'Points Allowed', 'Offense Expected Points', 'Defense  Expected Points', 'Special Teams  Expected Points'])




In [7]:
# Standardize the data
from sklearn.preprocessing import StandardScaler

# Correcting the list
sd_data = ['1st Downs', 'Total Yards Gained', ' Passing Yards', ' Rushing Yards', 'Passing Yards Allowed', 'Rushing Yards Allowed', 'Turnovers Lost',
           '1st Downs Allowed', 'Total Yards Allowed', 'Passing Yards Allowed', 'Rushing Yards Allowed',
           'Turnovers Gained', 'Prediction_LR', 'Prediction_ADA']


scaler = StandardScaler()
mlp_model_df[sd_data] = scaler.fit_transform(mlp_model_df[sd_data])

mlp_model_df.head()

,Team,Home,Points Scored,1st Downs,Total Yards Gained,Passing Yards,Rushing Yards,Turnovers Lost,1st Downs Allowed,Total Yards Allowed,Passing Yards Allowed,Rushing Yards Allowed,Turnovers Gained,Prediction_LR,Prediction_ADA
0,San Francisco 49ers,0,30,0.564721,0.691300,-0.212822,1.558456,-0.676373,-0.878539,-1.077223,-0.279643,-1.482330,0.396359,1.364103,1.728923
1,San Francisco 49ers,0,30,0.358541,0.388789,-0.172729,0.958573,-0.676373,1.801801,0.633125,1.043424,-0.489420,0.396359,1.364103,1.728923
2,San Francisco 49ers,1,30,1.389441,1.273051,1.083517,0.586232,-0.676373,-1.909439,-2.112740,-1.308695,-1.730558,-0.676373,1.364103,1.728923
3,San Francisco 49ers,1,35,2.214161,0.737840,0.695951,0.234576,-0.676373,0.770901,0.353884,0.508851,-0.158450,-0.676373,1.364103,1.728923
4,San Francisco 49ers,1,42,1.183261,1.040350,0.428665,1.186115,-0.676373,-2.321799,-1.565894,-1.054773,-1.151360,2.541823,1.364103,1.728923


In [8]:
mlp_model_df.dtypes

Team                      object
Home                       int64
Points Scored              int64
1st Downs                float64
Total Yards Gained       float64
 Passing Yards           float64
 Rushing Yards           float64
Turnovers Lost           float64
1st Downs Allowed        float64
Total Yards Allowed      float64
Passing Yards Allowed    float64
Rushing Yards Allowed    float64
Turnovers Gained         float64
Prediction_LR            float64
Prediction_ADA           float64
dtype: object

In [9]:
# Convert Team Names into boolean vlaues in the index
mlp_model_df = pd.get_dummies(mlp_model_df)


mlp_model_df.head()

,Home,Points Scored,1st Downs,Total Yards Gained,Passing Yards,Rushing Yards,Turnovers Lost,1st Downs Allowed,Total Yards Allowed,Passing Yards Allowed,...,Team_New Orleans Saints,Team_New York Giants,Team_New York Jets,Team_Philadelphia Eagles,Team_Pittsburgh Steelers,Team_San Francisco 49ers,Team_Seattle Seahawks,Team_Tampa Bay Buccaneers,Team_Tennessee Titans,Team_Washington Commanders
0,0,30,0.564721,0.691300,-0.212822,1.558456,-0.676373,-0.878539,-1.077223,-0.279643,...,False,False,False,False,False,True,False,False,False,False
1,0,30,0.358541,0.388789,-0.172729,0.958573,-0.676373,1.801801,0.633125,1.043424,...,False,False,False,False,False,True,False,False,False,False
2,1,30,1.389441,1.273051,1.083517,0.586232,-0.676373,-1.909439,-2.112740,-1.308695,...,False,False,False,False,False,True,False,False,False,False
3,1,35,2.214161,0.737840,0.695951,0.234576,-0.676373,0.770901,0.353884,0.508851,...,False,False,False,False,False,True,False,False,False,False
4,1,42,1.183261,1.040350,0.428665,1.186115,-0.676373,-2.321799,-1.565894,-1.054773,...,False,False,False,False,False,True,False,False,False,False


In [10]:
#MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Data processing
X = mlp_model_df.drop('Points Scored', axis=1).values
y = mlp_model_df['Points Scored'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25)

MLP_model = MLPRegressor(hidden_layer_sizes=100, max_iter= 5000, random_state= 40, batch_size= 30, solver= 'lbfgs', 
                        activation= 'identity')

MLP_model.fit(X_train, y_train)

y_pred = MLP_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1625: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


5.17050382397391


In [11]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    team1_data = mlp_model_df[nfl_df['Team'] == team1].drop('Points Scored', axis=1).reset_index(drop=True)
    team2_data = mlp_model_df[nfl_df['Team'] == team2].drop('Points Scored', axis=1).reset_index(drop=True)
    
    week_slice = slice(0,17)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opponent').columns
    
    team1_test[opp_columns] = 0
    team1_test['Team_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opponent').columns
    
    team2_test[opp_columns] = 0
    team2_test['Team_' + team1] = 1
    team2_test['Home'] = 1 # change to remove home field advantage
    
    # head to head matchup
    team1_test[['1st Downs Allowed','Total Yards Allowed','Passing Yards Allowed','Rushing Yards Allowed','Turnovers Gained']] = team2_test[['1st Downs','Total Yards Gained',' Passing Yards',' Rushing Yards','Turnovers Lost']]
    team2_test[['1st Downs Allowed','Total Yards Allowed','Passing Yards Allowed','Rushing Yards Allowed','Turnovers Gained']] = team1_test[['1st Downs','Total Yards Gained',' Passing Yards',' Rushing Yards','Turnovers Lost']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) # added to address the NANs that was causing the error
    
    scores = MLP_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

In [12]:
# pd.options.display.max_rows = None
# pd.options.display.max_columns = None

# def Score_Predictor1(home_team, away_team):
#     # Convert the index to strings
#     mlp_model_df.index = mlp_model_df.index.astype(str)
    
#     # Get data for the home team
#     team1_data = mlp_model_df[mlp_model_df.index.str.contains(home_team)].drop('Points Scored', axis=1).mean()
#     team1_test = pd.DataFrame(team1_data).T
#     team1_test.filter(like='Opponent').loc[:, :] = 0
#     team1_test['Team_' + away_team] = 1
#     team1_test['Home'] = 1
    
#     # Get data for the away team
#     team2_data = mlp_model_df[mlp_model_df.index.str.contains(away_team)].drop('Points Scored', axis=1).mean()
#     team2_test = pd.DataFrame(team2_data).T
#     team2_test.filter(like='Opponent').loc[:, :] = 0
#     team2_test['Team_' + home_team] = 1
#     team2_test['Home'] = 0
    
#     # Swap stats for head-to-head comparison
#     team1_test[['1st Downs Allowed', 'Total Yards Allowed', 'Passing Yards Allowed', 'Rushing Yards Allowed', 'Turnovers Gained']] = \
#     team2_test[['1st Downs', 'Total Yards Gained', ' Passing Yards', ' Rushing Yards', 'Turnovers Lost']]
    
#     team2_test[['1st Downs Allowed', 'Total Yards Allowed', 'Passing Yards Allowed', 'Rushing Yards Allowed', 'Turnovers Gained']] = \
#     team1_test[['1st Downs', 'Total Yards Gained', ' Passing Yards', ' Rushing Yards', 'Turnovers Lost']]
    
#     # Concatenate data for prediction
#     X_playoff_test = pd.concat([team1_test, team2_test])
#     X_playoff_test.fillna(0, inplace=True)
    
#     # Make predictions
#     scores = MLP_model.predict(X_playoff_test)
    
#     # Determine the winner
#     if scores[0] > scores[1]:
#         winner = home_team
#     else:
#         winner = away_team
    
#     print(f"{home_team} will score {round(scores[0], 1)}")
#     print(f"{away_team} will score {round(scores[1], 1)}")
#     print(f"The winner is: {winner}")
    
#     return scores, winner





# AFC Wild Card Round

In [13]:
scores, winner = Score_Predictor('Cleveland Browns', 'Houston Texans')

Cleveland Browns will score 24.5
Houston Texans will score 28.7
Houston Texans are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [14]:
scores, winner = Score_Predictor('Miami Dolphins', 'Kansas City Chiefs')

Miami Dolphins will score 27.9
Kansas City Chiefs will score 25.3
Miami Dolphins are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [15]:
scores, winner = Score_Predictor('Pittsburgh Steelers', 'Buffalo Bills')

Pittsburgh Steelers will score 22.0
Buffalo Bills will score 24.8
Buffalo Bills are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


# NFC Wild Card Round

In [16]:
scores, winner = Score_Predictor('Green Bay Packers', 'Dallas Cowboys')

Green Bay Packers will score 27.8
Dallas Cowboys will score 30.6
Dallas Cowboys are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [17]:
scores, winner = Score_Predictor('Los Angeles Rams', 'Detroit Lions')

Los Angeles Rams will score 27.1
Detroit Lions will score 27.9
Detroit Lions are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [18]:
scores, winner = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers')

c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


Philadelphia Eagles will score 26.0
Tampa Bay Buccaneers will score 24.7
Philadelphia Eagles are the WINNERS!!!


# AFC Divisional Round

In [19]:
scores, winner = Score_Predictor('Houston Texans', 'Baltimore Ravens')

Houston Texans will score 27.3
Baltimore Ravens will score 29.7
Baltimore Ravens are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [20]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'Buffalo Bills')

Kansas City Chiefs will score 26.3
Buffalo Bills will score 27.2
Buffalo Bills are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


# NFC Divisional Round

In [21]:
scores, winner = Score_Predictor('Green Bay Packers', 'San Francisco 49ers')

Green Bay Packers will score 24.4
San Francisco 49ers will score 28.3
San Francisco 49ers are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


In [22]:
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Detroit Lions')

Tampa Bay Buccaneers will score 22.6
Detroit Lions will score 26.0
Detroit Lions are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


# AFC Championship

In [23]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'Baltimore Ravens')

Kansas City Chiefs will score 27.0
Baltimore Ravens will score 29.4
Baltimore Ravens are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


# NFC Championship

In [24]:
scores, winner = Score_Predictor('Detroit Lions', 'San Francisco 49ers')

c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(


Detroit Lions will score 26.6
San Francisco 49ers will score 28.7
San Francisco 49ers are the WINNERS!!!


# SUPER BOWL LVIII (58)

In [26]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'San Francisco 49ers')

Kansas City Chiefs will score 23.2
San Francisco 49ers will score 27.0
San Francisco 49ers are the WINNERS!!!


c:\Users\james\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but MLPRegressor was fitted without feature names
  warnings.warn(
